# Parsing the Accumulated html Data

In [266]:
import os
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np


In [267]:
DATA_DIR = "data"

SCORES_DIR = os.path.join(DATA_DIR, "scores_stats") #scores and fixtures data
LEAGUE_DIR = os.path.join(DATA_DIR, "league_stats") #squad standard stats
MISC_DIR = os.path.join(DATA_DIR, "misc_stats")     #miscellaneous stats
PLAYING_DIR = os.path.join(DATA_DIR, "playing_stats")#playing time stats
KEEPER_DIR = os.path.join(DATA_DIR, "keeper_stats")  #keepers stats
SHOOTING_DIR = os.path.join(DATA_DIR, "shooting_stats") #shooting stats

years = list(range(2010,2022))


### Useful stuff:  
In case we could not do it with bs4. We have that header row in the middle of the data frame. Go to Venue column and delete any row that contains "Venue"

``` df = df[~df["Venue"].str.contains("Venue",na=False)]```


``` df.loc[df["Score_home"].isnull()]``` useful to find all the null values in a column

```df.loc[~df["Score"].str.startswith("("),"away_team_score"] = df["Score"].str[:1]``` i.e. ```df.loc[df['column name'] condition, 'new column name'] = 'value if condition is met'```

```df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)```

```games.drop(games[games['score']=="S"].index, inplace=True)``` drop rows inplace if the column contains char "S" in the score column

## Parsing the scores and fixtures table
This is the table that shows all the competitions in a season and the scores.

In [268]:
# Sorting out the directory and file name first:
score_fix = os.listdir(SCORES_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
score_fix = [os.path.join(SCORES_DIR,f) for f in score_fix if f.endswith(".html")] # this is a lit but we pass an element of it to the function


def parse_score(score_fix, year):

    # Parsing the html using beautiful soup
    with open (score_fix, encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    soup.find("tr", class_ = "thead").decompose() # get rid o the middle row that repeats the headers.

    [s.decompose() for s in soup.find_all("tr", class_ = "spacer partial_table result_all")] # deleting the table spacers, they give NaN value in pandas

    df = pd.read_html(str(soup))[0]
    # df = pd.read_html(score_fix[0])[0] # not using bs4

    # Cleaning the df, including splitting the score column and removing the irrelevant columns. We add a year column and remove Wk and Day columns. 
    df['year'] = year

    # some scores follow this formate e.g. 	(3) 1–1 (4)	We need to do proper processing for these. 


    # # score column cleaning
    df.loc[df["Score"].str.startswith("("),"home_team_score"] = df["Score"].str[1:2] # if the Score column starts with "(", then grab the second from left char.
    df.loc[~df["Score"].str.startswith("("),"home_team_score"] = df["Score"].str[:1] #If it does not start with "(" then do your normal thing and grab the left most char

    df.loc[df["Score"].str.startswith("("),"away_team_score"] = df["Score"].str[-2:-1] # if the Score column starts with "(", then grab the second from right char.
    df.loc[~df["Score"].str.startswith("("),"away_team_score"] = df["Score"].str[-1:] #If it does not start with "(" then do your normal thing and grab the right most char


    return(df)



In [269]:
df

,Round,Day,Date,Home,Score,Away,Venue,Referee,Match Report,year,home_team_score,away_team_score
0,Group stage,Tue,2010-09-14,Lyon fr,1â0,de Schalke 04,Stade de Gerland,Ivan Bebek,Match Report,2010,1,0
1,Group stage,Tue,2010-09-14,Manchester Utd eng,0â0,sct Rangers,Old Trafford,OlegÃ¡rio BenquerenÃ§a,Match Report,2010,0,0
2,Group stage,Tue,2010-09-14,Bursaspor tr,0â4,es Valencia,Bursa AtatÃ¼rk,Svein Oddvar Moen,Match Report,2010,0,4
3,Group stage,Tue,2010-09-14,Benfica pt,2â0,il Hapoel Tel Aviv,EstÃ¡dio do Sport Lisboa e Benfica,Aleksei Nikolaev,Match Report,2010,2,0
4,Group stage,Tue,2010-09-14,FC Copenhagen dk,1â0,ru Rubin Kazan,Telia Parken,Thomas Einwaller,Match Report,2010,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
120,Semi-finals,Tue,2011-04-26,Schalke 04 de,0â2,eng Manchester Utd,Arena AufSchalke,Velasco Carballo,Match Report,2010,0,2
121,Semi-finals,Wed,2011-04-27,Real Madrid es,0â2,es Barcelona,Estadio Santiago BernabÃ©u,Wolfgang Stark,Match Report,2010,0,2
122,Semi-finals,Tue,2011-05-03,Barcelona es,1â1,es Real Madrid,Camp Nou,Frank de Bleeckere,Match Report,2010,1,1
123,Semi-finals,Wed,2011-05-04,Manchester Utd eng,4â1,de Schalke 04,Old Trafford,Pedro ProenÃ§a,Match Report,2010,4,1


## Parsing the Squad Standard Stats (League Stats)
This is available in the league stats directory as tables.


In [270]:
# Sorting out the directory and file name first:
league_stats = os.listdir(LEAGUE_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
league_stats = [os.path.join(LEAGUE_DIR,f) for f in league_stats if f.endswith(".html")] # this is a lit but we pass an element of it to the function


def parse_league(league_stats,year):
    # Parsing the html using beautiful soup
    with open (league_stats, encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    [s.decompose() for s in soup.find_all("tr", class_ = "thead")] # get rid of the middle row that repeats the headers.
    soup.find("tr", class_ = "over_header").decompose() # remove the over_header

    df = pd.read_html(str(soup))[0] # columns like Poss,Min,90s, PreC, PreP are all null. We drop them

    df['year'] = year

    return (df)


## Parsing the Squad Play time stats

In [271]:
 # Sorting out the directory and file name first:
playing_stats = os.listdir(PLAYING_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
playing_stats = [os.path.join(PLAYING_DIR,f) for f in playing_stats if f.endswith(".html")] # this is a lit but we pass an element of it to the function

def parse_playing(playing_stats,year):   
    # Parsing the html using beautiful soup
    with open (playing_stats, encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    [s.decompose() for s in soup.find_all("tr", class_ = "thead")] # get rid of the middle row that repeats the headers.
    soup.find("tr", class_ = "over_header").decompose() # remove the over_header

    df = pd.read_html(str(soup))[0] # columns like Poss,Min,90s, PreC, PreP are all null. We drop them
    df.dropna(axis=1, inplace=True) #dropping 7 columns because NaN
    df['year'] = year
    return df


## Parsing the shooting stats 

In [272]:
# Sorting out the directory and file name first:
shooting_stats = os.listdir(SHOOTING_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
shooting_stats = [os.path.join(SHOOTING_DIR,f) for f in shooting_stats if f.endswith(".html")]# this is a lit but we pass an element of it to the function

def parse_shooting(shooting_stats,year):
    # Parsing the html using beautiful soup
    with open (shooting_stats, encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    [s.decompose() for s in soup.find_all("tr", class_ = "thead")] # get rid of the middle row that repeats the headers.
    soup.find("tr", class_ = "over_header").decompose() # remove the over_header

    df = pd.read_html(str(soup))[0] # columns like Poss,Min,90s, PreC, PreP are all null. We drop them
    df.dropna(axis=1, inplace=True) #dropping 7 columns because NaN
    df['year'] = year
    return df

## Parsing the keeper stats

In [273]:
# Sorting out the directory and file name first:
keeper_stats = os.listdir(KEEPER_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
keeper_stats = [os.path.join(KEEPER_DIR,f) for f in keeper_stats if f.endswith(".html")] # this is a lit but we pass an element of it to the function

def parse_keeper(keeper_stats,year):
    # Parsing the html using beautiful soup
    with open (keeper_stats[0], encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    [s.decompose() for s in soup.find_all("tr", class_ = "thead")] # get rid of the middle row that repeats the headers.
    soup.find("tr", class_ = "over_header").decompose() # remove the over_header

    df = pd.read_html(str(soup))[0] # columns like Poss,Min,90s, PreC, PreP are all null. We drop them
    df.dropna(axis=1, inplace=True) #dropping 7 columns because NaN
    df['year'] = year
    return df

## Parsing the Miscellaneous Stats

In [274]:
# Sorting out the directory and file name first:
misc_stats = os.listdir(MISC_DIR) #for scores and fixtures
# score-fix has only the file name, we need to join with the path too.
misc_stats = [os.path.join(MISC_DIR,f) for f in misc_stats if f.endswith(".html")]

def parse_misc(misc_stats, year):
    # Parsing the html using beautiful soup
    with open (misc_stats[0], encoding= 'unicode_escape') as f: #open file and read
        html = f.read()

    soup = BeautifulSoup(html,'html.parser') # instantiates bs4 using the file and html parser
    [s.decompose() for s in soup.find_all("tr", class_ = "thead")] # get rid of the middle row that repeats the headers.
    soup.find("tr", class_ = "over_header").decompose() # remove the over_header

    df = pd.read_html(str(soup))[0] # columns like Poss,Min,90s, PreC, PreP are all null. We drop them
    df.dropna(axis=1, inplace=True) #dropping 7 columns because NaN
    df['year'] = year
    return df 

# combining the team statistics

In [275]:
years = list(range(2010,2022))

In [276]:
score_summaries = [] #to append each year's stats
league_summaries = []
playing_summaries = []
shooting_summaries = []
keeper_summaries = []
misc_summaries = []

for year in years: #getting summary tables for all the yers

    score_df = parse_score(score_fix[years.index(year)], year) # passing the index of year to correctly get the stats from html files
    score_summaries.append(score_df)

    league_df = parse_league(league_stats[years.index(year)], year)
    league_summaries.append(league_df)

    misc_df = parse_league(misc_stats[years.index(year)], year)
    misc_summaries.append(misc_df)

    keeper_df = parse_league(keeper_stats[years.index(year)], year)
    keeper_summaries.append(keeper_df)

    shooting_df = parse_league(shooting_stats[years.index(year)], year)
    shooting_summaries.append(shooting_df)

    playing_df = parse_league(playing_stats[years.index(year)], year)
    playing_summaries.append(playing_df)


# concatenating and cleaning summary tables for each type of statistic
score_summary = pd.concat(score_summaries, axis = 0)
league_summary = pd.concat(league_summaries, axis = 0)
keeper_summary = pd.concat(keeper_summaries, axis = 0)
shooting_summary = pd.concat(shooting_summaries, axis = 0)
playing_summary = pd.concat(playing_summaries, axis = 0)
misc_summary = pd.concat(misc_summaries, axis = 0)



Out of the six tables we have, five of them are team statistics, and the score_summary is game based. Since the goal is to have stats in the game dataframe, we first combine all the stats per game per year (season) and then merge that with the score_summary, on home team, away team and year. From 2010 to 2022, the row count for all dataframes is 384 (makes sense as 32 teams per year * 12 years). SO the concatenation should go well. 

In [277]:
combined_stats = pd.concat([league_summary,playing_summary,keeper_summary,shooting_summary,misc_summary], axis = 1)

combined_stats.shape

(384, 120)

In [278]:
pd.DataFrame(combined_stats.isnull().sum()).T

,Squad,# Pl,Age,Poss,MP,Starts,Min,90s,Gls,Ast,...,Int,TklW,PKwon,PKcon,OG,year,Recov,Won,Lost,Won%
0,0,0,0,128,0,0,128,128,0,0,...,192,192,192,192,192,0,224,224,224,224


In [279]:
combined_stats.dropna(axis = 1, inplace=True) # We drop the null values because the constitute more than a third of the values in all the columns. 
combined_stats.shape

#combined_stats.rename(columns={'Squad': 'team', 'oldName2': 'newName2'}, inplace=True)

(384, 59)

In [280]:
# dropping duplicate columns such as Squad, year, etc. 

combined_stats = combined_stats.T.drop_duplicates().T

combined_stats.columns

Index(['Squad', '# Pl', 'Age', 'MP', 'Starts', 'Gls', 'Ast', 'G+A', 'G-PK',
       'PK', 'PKatt', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'year',
       'Min%', 'Subs', 'Mn/Sub', 'PPM', 'onG', 'onGA', '+/-', '# Pl', 'Min',
       'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%',
       'SoT', 'SoT/90', 'G/SoT', 'Fls'],
      dtype='object')

Encountered an issue. The namings for the teams are not consistent. team, team_opp and Squad have different way of naming. Otherwise it would be a nice simple left merge (or rather two left merges, as we want the opposing team stats too). So we make sure the team names are consistent across all the tables. 

In [281]:
combined_stats['Squad'] = combined_stats['Squad'].str[3:].str.lstrip() #The naming is not consistent, so remove the first 3 chars and then remove white spaces (prefixes such as eng, and sct have three letters as opposed ot two)

In [282]:
combined_stats['Squad'].unique()

array(['Ajax', 'Arsenal', 'Auxerre', 'Barcelona', 'Basel',
       'Bayern Munich', 'Benfica', 'Braga', 'Bursaspor', 'CFR Cluj',
       'Chelsea', 'FC Copenhagen', 'Hapoel Tel Aviv', 'Inter', 'Lyon',
       'Manchester Utd', 'Marseille', 'Milan', 'MÅ\xa0K Å½ilina',
       'Panathinaikos', 'Partizan', 'Rangers', 'Real Madrid', 'Roma',
       'Rubin Kazan', 'Schalke 04', 'Shakhtar', 'Spartak Moscow',
       'Tottenham', 'Twente', 'Valencia', 'Werder Bremen', 'APOEL FC',
       'BATE Borisov', 'CSKA Moscow', 'Dinamo Zagreb', 'Dortmund', 'Genk',
       'Leverkusen', 'Lille', 'Manchester City', 'Napoli', 'Olympiacos',
       'OÈ\x9belul GalaÈ\x9bi', 'Porto', 'Trabzonspor',
       'Viktoria PlzeÅ\x88', 'Villarreal', 'Zenit', 'Anderlecht',
       'Celtic', 'Dynamo Kyiv', 'Galatasaray', 'Juventus', 'MÃ¡laga',
       'Montpellier', 'NordsjÃ¦lland', 'Paris S-G', 'AtlÃ©tico Madrid',
       'Austria Wien', 'Real Sociedad', 'Steaua', 'Athletic Club',
       'Liverpool', 'Ludogorets', 'MalmÃ¶', 'Mona

In [283]:
combined_stats

,Squad,# Pl,Age,MP,Starts,Gls,Ast,G+A,G-PK,PK,...,Save%,W,D,L,CS,CS%,SoT,SoT/90,G/SoT,Fls
0,Ajax,17,24.2,10,66,6,4,10,6,0,...,77.8,2,5,3,1,10.0,27,2.7,0.22,81
1,Arsenal,25,24.8,8,88,20,13,33,17,3,...,74.4,5,0,3,1,12.5,46,5.75,0.37,130
2,Auxerre,20,27.3,8,66,3,2,5,3,0,...,58.6,1,1,6,8,100.0,20,2.5,0.15,76
3,Barcelona,25,26.6,13,143,30,24,54,28,2,...,75.7,8,4,1,5,38.5,98,7.54,0.29,134
4,Basel,18,25.7,10,66,8,6,14,8,0,...,60.7,5,1,4,1,10.0,32,3.2,0.25,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,Sporting CP,27,26.7,8,88,14,10,24,12,2,...,55.6,3,1,4,2,25.0,30,3.75,0.4,85
28,Villarreal,25,28.0,12,132,20,16,36,18,2,...,70.9,5,3,4,3,25.0,44,3.67,0.41,115
29,Wolfsburg,21,26.1,6,66,5,4,9,5,0,...,72.7,1,2,3,1,16.7,11,1.83,0.45,89
30,Young Boys,24,25.6,6,66,7,4,11,7,0,...,62.5,1,2,3,0,0.0,25,4.17,0.28,87


# cleaning the score statistics dataframe

In [284]:
score_summary.drop(['Notes','Score', 'Match Report', "Attendance","Round","Time","Referee","Day","Date","Venue","xG","xG.1"], axis = 1, inplace=True) # need data rich numeric values. 


In [285]:
score_summary

,Wk,Home,Away,year,home_team_score,away_team_score
0,1.0,Lyon fr,de Schalke 04,2010,1,0
1,1.0,Manchester Utd eng,sct Rangers,2010,0,0
2,1.0,Bursaspor tr,es Valencia,2010,0,4
3,1.0,Benfica pt,il Hapoel Tel Aviv,2010,2,0
4,1.0,FC Copenhagen dk,ru Rubin Kazan,2010,1,0
...,...,...,...,...,...,...
120,NaN,Manchester City eng,es Real Madrid,2021,4,3
121,NaN,Liverpool eng,es Villarreal,2021,2,0
122,NaN,Villarreal es,eng Liverpool,2021,2,3
123,NaN,Real Madrid es,eng Manchester City,2021,3,1


In [286]:
games = score_summary #games is easier to type

#renaming columns
games.rename(columns={'Home': 'team', 'Away': 'team_opp', 'home_team_score':'score', 'away_team_score':'score_opp'}, inplace=True)

# changing scores to numeric as opposed to string
games.drop(games[games['score']=="S"].index, inplace=True) #cleaning the rows where score is not a number
games[['score','score_opp']] = games[['score','score_opp']].astype("int").copy()

we need to add a result column, to show won, lost, and draw. (win as a flag = 1,0,2 for won, lost and drew respectively). We use numpy for this

In [288]:
conditions = [(games['score']>games['score_opp']),
              (games['score']<games['score_opp']),
              (games['score']==games['score_opp']),]

values = [1,0,2] # 1 is team wn, 0 is team lost, 2 is a draw

games["result"] = np.select(conditions,values)

# using string methods on the team name column to get consistent team names across all the columns. 
games['team'] = games['team'].str[:-3].str.rstrip() #making all the team names consistent to allow merging later
games['team_opp'] = games['team_opp'].str[3:].str.lstrip() #removing spaces form the left side  because of the naming inconsistency

# Merging the games data frame with the combines statistics

Game table is where the scores live
combined stats contains the statistics per season for each team. 

In [300]:
df2 = games.merge(combined_stats, how='left', left_on=['year','team'],right_on=["year","Squad"])

In [301]:
df2

,Wk,team,team_opp,year,score,score_opp,result,Squad,# Pl,Age,...,Save%,W,D,L,CS,CS%,SoT,SoT/90,G/SoT,Fls
0,1.0,Lyon,Schalke 04,2010,1,0,1,Lyon,20,25.5,...,68.2,3,2,3,2,25.0,44,5.5,0.25,105
1,1.0,Manchester Utd,Rangers,2010,0,0,2,Manchester Utd,28,27.7,...,87.3,6,6,1,8,61.5,68,5.23,0.25,166
2,1.0,Bursaspor,Valencia,2010,0,4,0,Bursaspor,21,28.0,...,55.6,0,1,5,0,0.0,20,3.33,0.1,76
3,1.0,Benfica,Hapoel Tel Aviv,2010,2,0,1,Benfica,19,25.6,...,50.0,2,0,4,1,16.7,28,4.67,0.25,102
4,1.0,FC Copenhagen,Rubin Kazan,2010,1,0,1,FC Copenhagen,19,26.6,...,82.5,3,5,4,3,25.0,21,1.75,0.24,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,NaN,Manchester City,Real Madrid,2021,4,3,1,Manchester City,26,27.2,...,58.8,7,2,3,4,33.3,65,5.27,0.4,126
1457,NaN,Liverpool,Villarreal,2021,2,0,1,Liverpool,28,27.3,...,50.0,10,1,2,4,30.8,79,6.08,0.34,151
1458,NaN,Villarreal,Liverpool,2021,2,3,0,Villarreal,25,28.0,...,70.9,5,3,4,3,25.0,44,3.67,0.41,115
1459,NaN,Real Madrid,Manchester City,2021,3,1,1,Real Madrid,25,28.0,...,80.6,9,0,4,5,38.5,66,4.83,0.38,120


In [305]:
#another merge for the opposing team stats: 

combined_stats_opp = combined_stats
combined_stats_opp.columns += "_opp"

In [307]:
df = df2.merge(combined_stats_opp, how='left', left_on=['year','team_opp'],right_on=["year_opp","Squad_opp"])

In [308]:
df # and this is the final product after all the cleaning, 1461 games, where we have the scores, and stats for the team and the opposing team. 

,Wk,team,team_opp,year,score,score_opp,result,Squad,# Pl,Age,...,Save%_opp,W_opp,D_opp,L_opp,CS_opp,CS%_opp,SoT_opp,SoT/90_opp,G/SoT_opp,Fls_opp
0,1.0,Lyon,Schalke 04,2010,1,0,1,Lyon,20,25.5,...,78.1,5,4,3,3,25.0,65,5.42,0.32,195
1,1.0,Manchester Utd,Rangers,2010,0,0,2,Manchester Utd,28,27.7,...,83.3,1,3,2,2,33.3,16,2.67,0.19,64
2,1.0,Bursaspor,Valencia,2010,0,4,0,Bursaspor,21,28.0,...,80.0,3,3,2,2,25.0,57,7.12,0.26,110
3,1.0,Benfica,Hapoel Tel Aviv,2010,2,0,1,Benfica,19,25.6,...,79.6,4,4,4,2,16.7,28,2.33,0.21,76
4,1.0,FC Copenhagen,Rubin Kazan,2010,1,0,1,FC Copenhagen,19,26.6,...,77.8,1,3,2,3,50.0,21,3.5,0.0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,NaN,Manchester City,Real Madrid,2021,4,3,1,Manchester City,26,27.2,...,80.6,9,0,4,5,38.5,66,4.83,0.38,120
1457,NaN,Liverpool,Villarreal,2021,2,0,1,Liverpool,28,27.3,...,70.9,5,3,4,3,25.0,44,3.67,0.41,115
1458,NaN,Villarreal,Liverpool,2021,2,3,0,Villarreal,25,28.0,...,50.0,10,1,2,4,30.8,79,6.08,0.34,151
1459,NaN,Real Madrid,Manchester City,2021,3,1,1,Real Madrid,25,28.0,...,58.8,7,2,3,4,33.3,65,5.27,0.4,126
